# MedMatch AI - Environment Test

This notebook verifies your development environment is working correctly.

**What we're testing:**
1. Python packages import correctly
2. PyTorch can access Mac Metal/MPS acceleration
3. Google AI API connection works
4. Basic medical entity extraction

## 1. Package Imports

In [ ]:
import sys
import os
from dotenv import load_dotenv

# Data processing
import pandas as pd
import numpy as np

# ML framework
import torch

# Google AI
import google.genai as genai

print(f"✓ Python {sys.version}")
print(f"✓ PyTorch {torch.__version__}")
print(f"✓ NumPy {np.__version__}")
print(f"✓ Pandas {pd.__version__}")

## 2. PyTorch Metal/MPS Test

In [ ]:
# Check MPS (Metal Performance Shaders) availability
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("✓ Metal/MPS acceleration: AVAILABLE")
    print(f"  Device: {device}")
    
    # Test tensor operations on MPS
    x = torch.randn(100, 100).to(device)
    y = torch.randn(100, 100).to(device)
    z = torch.matmul(x, y)
    print(f"  Test computation successful: {z.shape}")
else:
    print("⚠ Metal/MPS not available, falling back to CPU")
    device = torch.device("cpu")

## 3. Load Environment Variables

In [ ]:
# Load .env file from project root
load_dotenv()

api_key = os.getenv('GOOGLE_AI_API_KEY')
if api_key:
    masked = api_key[:10] + "..." + api_key[-4:]
    print(f"✓ API Key loaded: {masked}")
else:
    print("✗ API Key not found in .env")

## 4. Connect to Google AI

In [ ]:
# Create Google AI client
client = genai.Client(api_key=api_key)
print("✓ Connected to Google AI")

## 5. Test Medical Entity Extraction

Let's test with a sample patient record.

In [ ]:
# Sample medical record
sample_record = """
Patient: Jane Doe, DOB 07/22/1980, MRN: 87654321
Chief Complaint: Persistent headache
History: 44-year-old female with migraine history presents with severe 
bilateral throbbing headache, photophobia, and nausea. Duration 3 days.
Vitals: BP 128/82, HR 76, RR 16
Assessment: Migraine with aura
Plan: Sumatriptan 50mg, follow-up in 1 week
"""

print("Sample Record:")
print(sample_record)

In [ ]:
# Extract entities using Gemini
prompt = f"""Extract key medical entities from this record and format as JSON:

{sample_record}

Include: patient_info, chief_complaint, history, vitals, assessment, treatment_plan"""

response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=prompt
)

print("Extracted Entities:")
print(response.text)

## 6. Entity Matching Test

Test the core functionality: determining if two records refer to the same patient.

In [ ]:
# Two records that should match
record_a = "Jane M. Doe, 07/22/1980, migraine patient"
record_b = "Doe, Jane, born 7/22/80, history of migraines"

matching_prompt = f"""Compare these two patient records and determine if they refer to the same person.

Record A: {record_a}
Record B: {record_b}

Provide:
1. Match confidence (0-100%)
2. Matching factors
3. Discrepancies
4. Final recommendation (SAME / DIFFERENT / UNCERTAIN)"""

match_response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=matching_prompt
)

print("Matching Analysis:")
print(match_response.text)

## Summary

If all cells above ran successfully:
- ✓ Your environment is properly configured
- ✓ PyTorch can use Mac GPU acceleration
- ✓ Google AI API is working
- ✓ Medical entity extraction is functional
- ✓ Entity matching logic works

**You're ready to start developing MedMatch AI!**

### Next Steps:
1. Create sample patient datasets
2. Build entity extraction pipeline
3. Develop matching algorithms
4. Test with edge cases
5. Build confidence scoring system
6. Eventually switch to MedGemma for production